In [1]:
import os
import time
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

C:\Users\chuy-\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- CONFIGURACIÓN ---
MODEL_NAME = "llama3.2"
VECTOR_DB_PATH = "./chroma_db"
OUTPUT_FILE = "BORRADOR_INFORME.md"

In [3]:
PREGUNTAS = [
    "¿Qué expresiones o términos utiliza la Gen Z para describir el vacío existencial en redes sociales?",
    "¿Cómo influyen los algoritmos de recomendación en la construcción de su identidad?",
    "¿Qué emociones aparecen con mayor frecuencia cuando se habla de burnout o presión digital?",
    "¿La Gen Z percibe la autonomía como algo propio o como algo condicionado por la tecnología?",
    "¿Qué diferencias hay entre discursos auténticos vs discursos performativos en plataformas como TikTok?",
    "¿Existen patrones de lenguaje que indiquen crisis de sentido o desorientación vital?",
    "¿Cómo se refleja la idea de 'identidad líquida' en los datos recuperados?",
    "¿Qué menciones aparecen sobre libertad, control o manipulación algorítmica?",
    "¿Se observan señales de que los algoritmos crean deseos o hábitos?",
    "¿Qué temas o preocupaciones predominan en la conversación digital sobre propósito de vida?",
    "¿Hay evidencia de rechazo a los metarrelatos o valores tradicionales?",
    "¿Cómo aparece la figura del 'yo digital' en los textos analizados?",
    "¿Qué ejemplos concretos muestran pérdida del pensamiento crítico por efecto de la burbuja de filtros?",
    "¿Existen contrastes entre la visión que la Gen Z tiene de sí misma y lo que los datos sugieren?",
    "¿Qué rol juega la hiperconectividad en la ansiedad o depresión mencionada?",
    "¿Se observan patrones que apoyen las ideas de Byung-Chul Han sobre rendimiento y autoexplotación?",
    "¿Cómo interpretaría Foucault el régimen de vigilancia algorítmica detectado?",
    "¿Qué evidencias hay de que la tecnología 'desoculta' y transforma la vida según Heidegger?",
    "¿El espacio público digital está debilitado como afirma Habermas? ¿Qué muestran los datos?",
    "¿Cuáles son los principales miedos, frustraciones y esperanzas de la Gen Z frente al futuro?"
]

In [4]:
def cargar_vectorstore():
    print("📂 Cargando memoria del proyecto...")
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return Chroma(persist_directory=VECTOR_DB_PATH, embedding_function=embeddings)

In [5]:
def generar_reporte():
    vectorstore = cargar_vectorstore()
    retriever = vectorstore.as_retriever(search_kwargs={"k": 6}) # Traer más contexto (6 fragmentos)
    llm = ChatOllama(model=MODEL_NAME, temperature=0.4) # Un poco más creativo para la redacción

    # Prompt enfocado en redacción de informes
    template = """Eres un investigador académico analizando datos cualitativos.
    Usa el siguiente contexto (que mezcla teoría filosófica y comentarios reales de redes sociales) para responder la pregunta.
    
    Reglas:
    1. Tu respuesta debe ser analítica y formal.
    2. CONECTA SIEMPRE la teoría (Han, Bauman, Foucault) con la práctica (Comentarios Gen Z).
    3. Usa citas textuales breves de los comentarios si están disponibles en el contexto.
    4. Si no hay datos suficientes, indícalo claramente.

    Contexto recuperado: {context}

    Pregunta de investigación: {question}
    
    Respuesta detallada:"""
    
    prompt = ChatPromptTemplate.from_template(template)
    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # Crear/Limpiar archivo de salida
    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        f.write("# INFORME AUTOMATIZADO: GEN Z Y CRISIS DE SENTIDO\n")
        f.write(f"Generado por Llama 3.2 + RAG | Fecha: {time.strftime('%Y-%m-%d')}\n\n")

    print(f"🚀 Iniciando análisis de {len(PREGUNTAS)} preguntas de investigación...\n")

    for i, pregunta in enumerate(PREGUNTAS, 1):
        print(f"[{i}/{len(PREGUNTAS)}] Analizando: {pregunta} ...")
        
        try:
            respuesta = chain.invoke(pregunta)
            
            # Guardar en archivo paso a paso
            with open(OUTPUT_FILE, "a", encoding="utf-8") as f:
                f.write(f"## {i}. {pregunta}\n\n")
                f.write(respuesta + "\n\n")
                f.write("---\n\n")
                
        except Exception as e:
            print(f"❌ Error en pregunta {i}: {e}")

    print(f"\n✅ ¡Análisis completado! Abre el archivo '{OUTPUT_FILE}' para ver los resultados.")

In [6]:
generar_reporte()

📂 Cargando memoria del proyecto...

🚀 Iniciando análisis de 20 preguntas de investigación...

[1/20] Analizando: ¿Qué expresiones o términos utiliza la Gen Z para describir el vacío existencial en redes sociales? ...
[2/20] Analizando: ¿Cómo influyen los algoritmos de recomendación en la construcción de su identidad? ...
[3/20] Analizando: ¿Qué emociones aparecen con mayor frecuencia cuando se habla de burnout o presión digital? ...
[4/20] Analizando: ¿La Gen Z percibe la autonomía como algo propio o como algo condicionado por la tecnología? ...
[5/20] Analizando: ¿Qué diferencias hay entre discursos auténticos vs discursos performativos en plataformas como TikTok? ...
[6/20] Analizando: ¿Existen patrones de lenguaje que indiquen crisis de sentido o desorientación vital? ...
[7/20] Analizando: ¿Cómo se refleja la idea de 'identidad líquida' en los datos recuperados? ...
[8/20] Analizando: ¿Qué menciones aparecen sobre libertad, control o manipulación algorítmica? ...
[9/20] Analizando: